In [16]:
import pandas as pd
from apyori import apriori
ordersdata = pd.read_csv("/Users/macbookpro/Documents/ds_intern_coding_challenge/orders.csv")
ordersdatacopy = pd.read_csv("/Users/macbookpro/Documents/ds_intern_coding_challenge/orders.csv")
ordersdata1 = pd.read_csv("/Users/macbookpro/Documents/ds_intern_coding_challenge/order_lines.csv")
ordersdata1copy = pd.read_csv("/Users/macbookpro/Documents/ds_intern_coding_challenge/order_lines.csv")
productsdata = pd.read_csv("/Users/macbookpro/Documents/ds_intern_coding_challenge/products.csv")



In [17]:
ordersdata.head()

,order_id,metro_id,store_id,customer_id,created_at
0,13571562,1,1,792079,2019-01-02 23:40:32.385831
1,13655794,1,1,1903795,2019-01-05 20:11:05.570093
2,14123099,1,1,127956,2019-01-19 19:57:43.865203
3,14420509,1,1,1012340,2019-01-27 22:26:18.602900
4,14208064,20,1,93594,2019-01-21 21:46:26.607809


In [18]:
ordersdata1.head()

,order_id,product_id
0,13571562,10519
1,13571562,16492
2,13571562,8438
3,13571562,24082
4,13571562,11086


In [19]:
ordersdata1["order_id"]

0         13571562
1         13571562
2         13571562
3         13571562
4         13571562
            ...   
301047    14442540
301048    14442540
301049    14442540
301050    14442540
301051    14442540
Name: order_id, Length: 301052, dtype: int64

In [20]:
#data understanding question 1 code to join two datasets
ordersdata1 = ordersdata1.dropna(axis=1)
merged = ordersdata.merge(ordersdata1, on='order_id')
merged.to_csv("question1.csv", index=False)

In [32]:
#data understanding question 2
comb = ordersdatacopy['order_id'].isin(ordersdata1copy['order_id'])
ordersdatacopy.drop(ordersdatacopy[comb].index, inplace = True)
#creates new csv for question 2 with only order ids in orders.csv 
ordersdatacopy.to_csv("question2.csv", index=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [23]:
#question 3 code to join dataset with product names
merged1 = merged.merge(productsdata, on='product_id')
merged1.to_csv("question3.csv", index=False)

In [24]:
merged1.head()


,order_id,metro_id,store_id,customer_id,created_at,product_id,name
0,13571562,1,1,792079,2019-01-02 23:40:32.385831,10519,Stubb's Sticky Sweet BBQ Sauce
1,13832114,1,1,173225,2019-01-11 04:13:27.542481,10519,Stubb's Sticky Sweet BBQ Sauce
2,14013191,1,1,66992,2019-01-16 14:28:26.121686,10519,Stubb's Sticky Sweet BBQ Sauce
3,14454936,1,1,52744,2019-01-28 22:11:03.606890,10519,Stubb's Sticky Sweet BBQ Sauce
4,13571562,1,1,792079,2019-01-02 23:40:32.385831,16492,Hamburger Buns


In [25]:
#you can see top 10 item names here
#this is important for question 3 in data understanding part a
merged1['name'].value_counts()[0:10]

Banana                               5929
Strawberries                         3040
Hass Avocado                         2993
2% Reduced Fat Milk                  2068
Red Seedless Grapes                  1970
Large Grade A Eggs                   1798
Whole Milk                           1629
Publix 93% Lean Ground Beef          1529
Boneless Skinless Chicken Breasts    1494
Honey Wheat Bread                    1321
Name: name, dtype: int64

In [58]:
#question 3 part b
#group by number of product_ids that are unique
getstats=ordersdata1.groupby('order_id')['product_id'].nunique()
#getstats=ordersdata1.groupby('order_id').agg({'product_id': lambda x: len(set(x))})
getstats.head()

,product_id
order_id,
13516253,26
13516305,7
13516331,9
13516346,11
13516380,8


In [28]:
getstats.to_csv("getstats.csv", index=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [29]:
#this will give all of the data necessary for question 3 part b 
getstats.describe()

count    16464.000000
mean        18.285471
std         10.562312
min          1.000000
25%         11.000000
50%         16.000000
75%         23.000000
max        127.000000
Name: product_id, dtype: float64

In [33]:
#test to see if part 2 worked by seeing difference from ordersdata
ordersdatacopy.head()

,order_id,metro_id,store_id,customer_id,created_at
4,14208064,20,1,93594,2019-01-21 21:46:26.607809
19,14364287,20,1,549691,2019-01-26 21:42:56.121338
36,13846253,20,1,1632766,2019-01-11 18:35:55.123202
42,14157491,20,1,1260227,2019-01-20 18:39:02.779142
55,13908958,20,1,755897,2019-01-13 15:41:23.591135


In [69]:
#starting apriori
dataapyrori=merged1.groupby('order_id')

In [70]:
#processing data 
transactions = merged1.groupby('order_id').agg({'name':lambda x: list(x)})

In [77]:
#get rules
rules = apriori(transactions['name'].tolist(), min_support = 0.003, min_confidence=0.001, min_lift=0.01, max_length = 3)

In [78]:
results = list(rules)

In [79]:
#get length I was not able to get to 15000 because it was not computing fast enough
print(len(results))

4050


In [80]:
#testing for item sets
results[-1].ordered_statistics

[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Strawberries', 'Red Seedless Grapes', 'Zucchini Squash'}), confidence=0.004312439261418853, lift=1.0),
 OrderedStatistic(items_base=frozenset({'Red Seedless Grapes'}), items_add=frozenset({'Strawberries', 'Zucchini Squash'}), confidence=0.03604060913705583, lift=2.412083694441005),
 OrderedStatistic(items_base=frozenset({'Strawberries'}), items_add=frozenset({'Red Seedless Grapes', 'Zucchini Squash'}), confidence=0.023355263157894737, lift=2.3025212732429874),
 OrderedStatistic(items_base=frozenset({'Zucchini Squash'}), items_add=frozenset({'Strawberries', 'Red Seedless Grapes'}), confidence=0.07244897959183673, lift=1.7697329376854598),
 OrderedStatistic(items_base=frozenset({'Strawberries', 'Red Seedless Grapes'}), items_add=frozenset({'Zucchini Squash'}), confidence=0.10534124629080117, lift=1.7697329376854598),
 OrderedStatistic(items_base=frozenset({'Red Seedless Grapes', 'Zucchini Squash'}), items_add=frozenset({'Str

In [81]:
# once we have the statistics and rules we use the following code to visualize the data
df = pd.DataFrame(columns=('order_id','name','Consequent','Support','Confidence','Lift'))

Support =[]
Confidence = []
Lift = []
Items = []
Antecedent = []
Consequent=[]

counter = 0
for RelationRecord in results:
    for ordered_stat in RelationRecord.ordered_statistics:
        Support.append(RelationRecord.support)
        Items.append(RelationRecord.items)
        Antecedent.append(ordered_stat.items_base)
        Consequent.append(ordered_stat.items_add)
        Confidence.append(ordered_stat.confidence)
        Lift.append(ordered_stat.lift)
        counter += 1
print(counter)

df['Items'] = list(map(set, Items))                                   
df['Antecedent'] = list(map(set, Antecedent))
df['Consequent'] = list(map(set, Consequent))
df['Support'] = Support
df['Confidence'] = Confidence
df['Lift']= Lift

12848


In [82]:
#find highest support items
df.sort_values(by ='Support', ascending = False, inplace = True)

In [83]:
#visualize the support data to find items
df.head(15)

,order_id,name,Consequent,Support,Confidence,Lift,Items,Antecedent
86,NaN,NaN,{Banana},0.360119,0.360119,1.000000,{Banana},{}
1033,NaN,NaN,{Strawberries},0.184645,0.184645,1.000000,{Strawberries},{}
478,NaN,NaN,{Hass Avocado},0.181791,0.181791,1.000000,{Hass Avocado},{}
24,NaN,NaN,{2% Reduced Fat Milk},0.124271,0.124271,1.000000,{2% Reduced Fat Milk},{}
879,NaN,NaN,{Red Seedless Grapes},0.119655,0.119655,1.000000,{Red Seedless Grapes},{}
539,NaN,NaN,{Large Grade A Eggs},0.108601,0.108601,1.000000,{Large Grade A Eggs},{}
1164,NaN,NaN,{Whole Milk},0.097182,0.097182,1.000000,{Whole Milk},{}
3097,NaN,NaN,"{Banana, Strawberries}",0.096939,0.096939,1.000000,"{Banana, Strawberries}",{}
3098,NaN,NaN,{Strawberries},0.096939,0.269185,1.457851,"{Banana, Strawberries}",{Banana}
3099,NaN,NaN,{Banana},0.096939,0.525000,1.457851,"{Banana, Strawberries}",{Strawberries}


In [84]:
len(df)

12848

In [89]:
#change this to find item sets of certain length
print(len(df[df['Items'].apply(lambda x: len(x)==1)]))

1189
